### Make sure to run in the ATLAS controls ACS023 terminal via controlsys login: python3.6 api_server.py

## Start of good code to run loops to collect data

In [1]:
#settings / loading
import requests
import pandas as pd
import os
from random import random
import time
write2database = True #write to influx?
post_settings = False #whether to send new magnet settings
setting_list = ["QTA800:CONTROL_Y1","QTA800:CONTROL_X1","QTA800:CONTROL_Y2",
                "STA801:CONTROL_X","STA801:CONTROL_Y",
                "QDA800:CONTROL_X","QDA800:CONTROL_Y",
                "MGA002:CONTROL",
                "QDA002:CONTROL_X","QDA002:CONTROL_Y",
                "STA004:CONTROL_X","STA004:CONTROL_Y",
                "QDA001:CONTROL_X","QDA001:CONTROL_Y",
                "MGA001:CONTROL",
                "QTA001:CONTROL_X1","QTA001:CONTROL_X2","QTA001:CONTROL_Y",
                "STA003:CONTROL_X","STA003:CONTROL_Y",
                "STA002:CONTROL_X","STA002:CONTROL_Y",
                "STA001:CONTROL_X","STA001:CONTROL_Y"] #which magnets to read
#setting_list = ["R501:PHASEB","R501:CONTROL_AMP"]
max_vals = [10.0,15.0,20.0,1.5,1.5] #max field setting for bl magnets
min_vals = [0.5,0.5,0.5,-1.5,-1.5]
off_vals = [0.0,0.0,0.0,-1.1,-1.1]
delta_vals = [0.5,0.5,0.5,0.02,0.02] #amount by which to change 
orig_vals = [4.051278,5.904756,5.272509,-1.036212,0.827078] #initial tune settings before scans
num_loops = 10000 #finite numer of samples if using
num_samples = 2 #times to measure the quads before changing them # not used at present

In [2]:
#initial read and need to save in df idealy
for loops in range(1):
    print("On Loop Number: ",loops)
    i=0
    for names in setting_list:
    #print(names)
        #read present setting
        if (write2database):
            #for names in setting_list:
            #read new settings
            resp = requests.get('http://146.139.66.23:5000/', json={
            "setting": [names],
            "fcup": [],
            "bpm": [] })
            df = pd.json_normalize(resp.json())
            temp_cntrl_name = "setting.control." + names
            temp_mon_name = "setting.monitor." + names
            print("Monitor: ",temp_mon_name,df[temp_mon_name].iat[0])
            dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_cntrl_name,df[temp_cntrl_name].iat[0])
            os.system(dbwrite)
            dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_mon_name,df[temp_mon_name].iat[0])
            os.system(dbwrite)
    time.sleep(5) #time to see detector rates

On Loop Number:  0
Monitor:  setting.monitor.QTA800:CONTROL_Y1 3.0183119773864746
Monitor:  setting.monitor.QTA800:CONTROL_X1 6.0586018562316895
Monitor:  setting.monitor.QTA800:CONTROL_Y2 7.1479644775390625
Monitor:  setting.monitor.STA801:CONTROL_X 0.39797067642211914
Monitor:  setting.monitor.STA801:CONTROL_Y 1.1516659259796143
Monitor:  setting.monitor.QDA800:CONTROL_X 5.738879680633545
Monitor:  setting.monitor.QDA800:CONTROL_Y 9.428159713745117
Monitor:  setting.monitor.MGA002:CONTROL 24.809999465942383
Monitor:  setting.monitor.QDA002:CONTROL_X 3.989579916000366
Monitor:  setting.monitor.QDA002:CONTROL_Y 3.797499895095825
Monitor:  setting.monitor.STA004:CONTROL_X -1.1437220573425293
Monitor:  setting.monitor.STA004:CONTROL_Y -0.6343321800231934
Monitor:  setting.monitor.QDA001:CONTROL_X 6.831620216369629
Monitor:  setting.monitor.QDA001:CONTROL_Y 5.016510009765625
Monitor:  setting.monitor.MGA001:CONTROL -1.2949999570846558
Monitor:  setting.monitor.QTA001:CONTROL_X1 11.9750661

In [ ]:
# constant running loop with option to modify and read
#right now changes one element at a time then reads
for loops in range(num_loops):
    print("On Loop Number: ",loops)
    i=0
    for names in setting_list:
    #print(names)
        #read present setting
        if (post_settings):
            resp = requests.get('http://146.139.66.23:5000/', json={
            "setting": [names],
            "fcup": [],
            "bpm": [] })
            df = pd.json_normalize(resp.json())
            temp_cntrl_name = "setting.control." + names
            temp_mon_name = "setting.monitor." + names
            new = 0.0
            delta = 0.0
            #calculate new setting and post it
            delta=random()*delta_vals[i]#could be modified
            if random() < 0.55:
                new = df[temp_mon_name].iat[0] + delta #use monitor value
            else:
                new = df[temp_mon_name].iat[0] - delta #use monitor value
            #check to make sure within min/max
            if new > max_vals[i] or new < min_vals[i]:
                new = orig_vals[i]
            if loops%40 == 0:
                new = orig_vals[i]
            dict_post = {names : new}
            print("Dictionary Post: ",dict_post)
            # uncomment below for actual post!!!
            #resp_post = requests.post('http://146.139.66.23:5000/', json=dict_post)
            i=i+1
            time.sleep(2) #wait after post

        if (write2database):
            #for names in setting_list:
            #read new settings
            resp = requests.get('http://146.139.66.23:5000/', json={
            "setting": [names],
            "fcup": [],
            "bpm": [] })
            df = pd.json_normalize(resp.json())
            temp_cntrl_name = "setting.control." + names
            temp_mon_name = "setting.monitor." + names
            #print("Monitor: ",temp_mon_name,df[temp_mon_name].iat[0])
            dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_cntrl_name,df[temp_cntrl_name].iat[0])
            os.system(dbwrite)
            dbwrite = "influx -execute \'insert bl,tag={} value={}\' -database=db".format(temp_mon_name,df[temp_mon_name].iat[0])
            os.system(dbwrite)
    time.sleep(5) #time to see detector rates